# Quick Start 

Welcome to the **ASL(Agent Stucture Language)** — A DSL(Domain Specific Language) about how to building an agent — documentation! This page will give you an introduction to 80% of the ASL usage that you will use on a daily basis.

> You will learn:
>   1. How to build an agent by ASL.
>   2. How to reuse the exit agents in a componentized parttern.
>   3. How to build an agent with a nested structure.
>   4. How to control data transmission in the agent
>   5. How to achieve dynamic topology during the agent's runtime.


## Introduction

ASL is a declarative language for agent construction. After all basic functions are modularly implemented, we pursue a what-you-see-is-what-you-get approach for the internal processes of the agent. We can clearly see the orchestration process and hierarchical structure at a glance.

### Build an Agent by ASL

Take the simplest text generation process as an example. When a user inputs a query, we break it down, and then generate text for every sub-query.

Let's first prepare necessary environment variables.

In [ ]:
# Get the environment variables.
import os

_api_key = os.environ.get("OPENAI_API_KEY")
_api_base = os.environ.get("OPENAI_API_BASE")
_model_name = os.environ.get("OPENAI_MODEL_NAME")

# Import the necessary packages.
from typing import List, Dict
from bridgic.core.model.types import Message, Role
from bridgic.core.agentic.asl import ASLAutoma, graph
from bridgic.llms.openai import OpenAILlm, OpenAIConfiguration

llm = OpenAILlm(  # the llm instance
    api_base=_api_base,
    api_key=_api_key,
    timeout=5,
    configuration=OpenAIConfiguration(model=_model_name),
)

Secondly, modularly implement the functional functions needed in this process.

In [39]:
# Break down the query into a list of sub-queries.
async def break_down_query(user_input: str) -> List[str]:
    llm_response = await llm.achat(
        messages=[
            Message.from_text(text=f"Break down the query into multiple sub-queries and only return the sub-queries", role=Role.SYSTEM),
            Message.from_text(text=user_input, role=Role.USER,),
        ]
    )
    return [item.strip() for item in llm_response.message.content.split("\n") if item.strip()]

# Define the function to conduct a web search.
async def query_answer(queries: List[str]) -> Dict[str, str]:
    answers = []
    for query in queries:
        response = await llm.achat(
            messages=[
                Message.from_text(text=f"Answer the given query briefly", role=Role.SYSTEM),
                Message.from_text(text=query, role=Role.USER,),
            ]
        )
        answers.append(response.message.content)
    
    res = {
        query: answer
        for query, answer in zip(queries, answers)
    }
    return res

Now, Let's complete this process using ASL.

In [40]:
class SplitSolveAgent(ASLAutoma):
    with graph as g:
        a = break_down_query
        b = query_answer

        +a >> ~b

The implementation process of `SplitSolveAgent` was accomplished through orchestration using ASL grammar. In this grammar:
- `with graph as g`: Represents opening a graph, and we can **declare the nodes** and **defining the dependency** between the nodes under its syntax block.
- `a = break_down_query`: Represents declaring a node names `a`.
- `a >> b`: Represents defining the dependency of `b` is `a`, which means the `b` will execute after `a`.
- `+a`: Represents defining `a` is the start.
- `~b`: Represents defining `b` is the output.

Now, Let's run it!

In [ ]:
text_generation_agent = SplitSolveAgent()
await text_generation_agent.arun("When and where was the Einstein born?")

{'1. When was Einstein born?': 'Albert Einstein was born in 1879.',
 '2. Where was Einstein born?': 'Albert Einstein was born in Ulm, Kingdom of Württemberg, German Empire.'}

Great! We successfully obtained the result. In the ASL code, we can see very intuitively that the `SplitSolveAgent` has only two nodes, and `b` depends on `a`, with no other redundant information.

### Reuse the Exit Agents in Componentized Parttern

In the above process, we have completed the agent that splits the query and answers them separately. It is a pre-designed module. Now, I want to design a chatbot that merge these individual answers to generate a unified response to the original question. Like this:

In [41]:
async def merge_answers(qa_pairs: Dict[str, str], user_input: str) -> str:
    answers = "\n".join([v for _, v in qa_pairs.values()])
    llm_response = await llm.achat(
        messages=[
            Message.from_text(text=f"Merge the given answers into a unified response to the original question", role=Role.SYSTEM),
            Message.from_text(text=f"Query: {user_input}\nAnswers: {answers}", role=Role.USER,),
        ]
    )
    return llm_response.message.content

# Define the Chatbot agent, use SplitSolveAgent in componentized parttern.
class Chatbot(ASLAutoma):
    with graph as g:
        a = SplitSolveAgent()
        b = merge_answers

        +a >> ~b

Let's run it.

In [42]:
chatbot = Chatbot()
await chatbot.arun(user_input="When and where was the Einstein born?")

TypeError: break_down_query() missing 1 required positional argument: 'user_input'